Goal: Create a daily NAO index using the CPC methodology 
=====

Author: [Ray Bell](https://github.com/raybellwaves)

Use the time period of the [SubX](http://iridl.ldeo.columbia.edu/SOURCES/.Models/.SubX/) hindcast: 1999-2016.
For a saninty check make sure that 1999-2016 is indeed the hindcast period:

In [3]:
import xarray as xr
remote_data = xr.open_dataset('http://iridl.ldeo.columbia.edu/SOURCES/.Models'\
                              '/.SubX/.RSMAS/.CCSM4/.hindcast/.zg/dods')
print(remote_data.coords['S'])

<xarray.DataArray 'S' (S: 6569)>
array(['1999-01-07T00:00:00.000000000', '1999-01-08T00:00:00.000000000',
       '1999-01-09T00:00:00.000000000', ..., '2016-12-29T00:00:00.000000000',
       '2016-12-30T00:00:00.000000000', '2016-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * S        (S) datetime64[ns] 1999-01-07 1999-01-08 1999-01-09 1999-01-10 ...
Attributes:
    long_name:      Start Time
    pointwidth:     0.0
    standard_name:  forecast_reference_time
    gridtype:       0


The data used here is geopotential height at 500 hPa ($Z_{500}$) from ERA-Interim. This data was simply copied from the University of Reading archive. You can however use the [API](https://software.ecmwf.int/wiki/display/CKB/How+to+download+data+via+the+ECMWF+WebAPI) to download the data from ECMWF.

Two files exists in this directory `a.nc` and `b.nc`. `a.nc` is the monthly mean of $Z_{500}$ January 1996 - December 2016. `b.nc` is daily $Z_{500}$ January 1st 1996 - December 31st 2016. The data has been interpolated to 1 degree to match the SubX data using cdo...